In [112]:
from statsbombpy import sb
import pandas as pd
import numpy as np
import networkx as nx
from collections import Counter

In [113]:
events = sb.competition_events(
    country="Germany",
    division= "1. Bundesliga",
    season="2023/2024",
    gender="male"
)

C:\Users\thors\AppData\Roaming\Python\Python312\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [114]:
#make a list to hold a df of the events of a match
df_dict = {}

for id in events.match_id.unique():
    #find one match for one team
    match_subset = events.loc[events['match_id'] == id]

    #we identify the starting 11
    starting_11 = match_subset.loc[match_subset['type'] == 'Starting XI'].loc[match_subset['team'] == 'Bayer Leverkusen', 'tactics'].to_list()[0]
    position_dict = {}
    
    #we make a dictionary for positions of players
    for member in starting_11['lineup']:
        player_id = int(member['player']['id'])
        position_name = member['position']['name']
        position_dict[player_id] = position_name


    #include only passes, shots, substitution or tactical shifts
    match_subset = match_subset.loc[(match_subset['type'].isin(['Shot','Pass','Substitution', 'Tactical Shift']))]

    #We include rows with Leverkusen as possession team or with type substitution or tactical shift
    match_subset = match_subset.loc[(match_subset['possession_team'] == 'Bayer Leverkusen') | (match_subset['type'].isin(['Substitution', 'Tactical Shift']))]

    #sort the values like when we did the passing sequences
    match_subset = match_subset.sort_values(['period','timestamp'], ascending=[True, True])
        
    match_subset['pass_recipient_position'] = np.nan

    for index, row in match_subset.iterrows():
        #If substitution, we update the dictionary to include player
        if row['type'] == 'Substitution' and row['team'] == 'Bayer Leverkusen':
            position_dict[row['substitution_replacement_id']] = row['position']

        #In case of a tactical shift, create a new position_dict
        if row['type'] == 'Tactical Shift' and row['team'] == 'Bayer Leverkusen':
            lineup = row['tactics']
            position_dict = {}
            for member in lineup['lineup']:
                player_id = int(member['player']['id'])
                position_name = member['position']['name']
                position_dict[player_id] = position_name

        elif row['pass_recipient_id'] in list(position_dict.keys()):
            #if the player is in our dictionary, we assign his position - else he may be a player of the opponent, and we ignore him
            match_subset.at[index, 'pass_recipient_position'] = position_dict[int(row['pass_recipient_id'])]
            
    #den næste linje kan eventuelt kommenteres ud, og så kan vi bare fjerne na i stedet, hvis vi ikke vil have de ekstra nodes, jeg foreslår :D
    #add shot as a pass recipient as well
    #match_subset.loc[match_subset['type'] =='Shot', 'pass_recipient_position'] = match_subset.apply(lambda x: np.where(pd.isna(x['pass_recipient_position']), x['shot_outcome'], x['pass_recipient_position']), axis=1)

    #if a does not have a pass_recipient_position yet, we drop it
    match_subset.dropna(subset=['pass_recipient_position'], inplace = True)
    df_dict[id] = match_subset 


C:\Users\thors\AppData\Local\Temp\ipykernel_9208\3895806944.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  match_subset['pass_recipient_position'] = np.nan
C:\Users\thors\AppData\Local\Temp\ipykernel_9208\3895806944.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Goalkeeper' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  match_subset.at[index, 'pass_recipient_position'] = position_dict[int(row['pass_recipient_id'])]
C:\Users\thors\AppData\Local\Temp\ipykernel_9208\3895806944.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performan

In [115]:
from sklearn.preprocessing import MinMaxScaler

for i in df_dict:
    m = df_dict[i]
    m[['x', 'y']] = m['location'].apply(pd.Series)

    # Step 2: Calculate average x and y for each position
    average_locations = m.groupby('position')[['x', 'y']].transform('mean')

    # Step 3: Assign average x and y values to new columns
    m['avg_x'] = average_locations['x']
    m['avg_y'] = average_locations['y']

    # Step 4: Normalize avg_x and avg_y
    scaler = MinMaxScaler()
    m[['avg_x', 'avg_y']] = scaler.fit_transform(m[['avg_x', 'avg_y']])

    # Update the dictionary
    df_dict[i] = m

C:\Users\thors\AppData\Local\Temp\ipykernel_9208\2048105922.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  m[['x', 'y']] = m['location'].apply(pd.Series)
C:\Users\thors\AppData\Local\Temp\ipykernel_9208\2048105922.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  m[['x', 'y']] = m['location'].apply(pd.Series)
C:\Users\thors\AppData\Local\Temp\ipykernel_9208\2048105922.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Co

In [116]:
#make a list to hold a df of the events of a match
opp_df_dict = {}

for id in events.match_id.unique():
    #find one match for one team
    match_subset = events.loc[events['match_id'] == id]

    #we identify the starting 11
    starting_11 = match_subset.loc[match_subset['type'] == 'Starting XI'].loc[match_subset['team'] != 'Bayer Leverkusen', 'tactics'].to_list()[0]
    position_dict = {}
    
    #we make a dictionary for positions of players
    for member in starting_11['lineup']:
        player_id = int(member['player']['id'])
        position_name = member['position']['name']
        position_dict[player_id] = position_name


    #include only passes, shots, substitution or tactical shifts
    match_subset = match_subset.loc[(match_subset['type'].isin(['Shot','Pass','Substitution', 'Tactical Shift']))]

    #We include rows with Leverkusen as possession team or with type substitution or tactical shift
    match_subset = match_subset.loc[(match_subset['possession_team'] != 'Bayer Leverkusen') | (match_subset['type'].isin(['Substitution', 'Tactical Shift']))]

    #sort the values like when we did the passing sequences
    match_subset = match_subset.sort_values(['period','timestamp'], ascending=[True, True])
        
    match_subset['pass_recipient_position'] = np.nan

    for index, row in match_subset.iterrows():
        #If substitution, we update the dictionary to include player
        if row['type'] == 'Substitution' and row['team'] != 'Bayer Leverkusen':
            position_dict[row['substitution_replacement_id']] = row['position']

        #In case of a tactical shift, create a new position_dict
        if row['type'] == 'Tactical Shift' and row['team'] != 'Bayer Leverkusen':
            lineup = row['tactics']
            position_dict = {}
            for member in lineup['lineup']:
                player_id = int(member['player']['id'])
                position_name = member['position']['name']
                position_dict[player_id] = position_name

        elif row['pass_recipient_id'] in list(position_dict.keys()):
            #if the player is in our dictionary, we assign his position - else he may be a player of the opponent, and we ignore him
            match_subset.at[index, 'pass_recipient_position'] = position_dict[int(row['pass_recipient_id'])]
            
    #den næste linje kan eventuelt kommenteres ud, og så kan vi bare fjerne na i stedet, hvis vi ikke vil have de ekstra nodes, jeg foreslår :D
    #add shot as a pass recipient as well
    #match_subset.loc[match_subset['type'] =='Shot', 'pass_recipient_position'] = match_subset.apply(lambda x: np.where(pd.isna(x['pass_recipient_position']), x['shot_outcome'], x['pass_recipient_position']), axis=1)

    #if a does not have a pass_recipient_position yet, we drop it
    match_subset.dropna(subset=['pass_recipient_position'], inplace = True)
    opp_df_dict[id] = match_subset 


C:\Users\thors\AppData\Local\Temp\ipykernel_9208\240695519.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  match_subset['pass_recipient_position'] = np.nan
C:\Users\thors\AppData\Local\Temp\ipykernel_9208\240695519.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Left Center Forward' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  match_subset.at[index, 'pass_recipient_position'] = position_dict[int(row['pass_recipient_id'])]
C:\Users\thors\AppData\Local\Temp\ipykernel_9208\240695519.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor per

In [118]:
from sklearn.preprocessing import MinMaxScaler

for i in opp_df_dict:
    m = df_dict[i]
    m[['x', 'y']] = m['location'].apply(pd.Series)

    # Step 2: Calculate average x and y for each position
    average_locations = m.groupby('position')[['x', 'y']].transform('mean')

    # Step 3: Assign average x and y values to new columns
    m['avg_x'] = average_locations['x']
    m['avg_y'] = average_locations['y']

    # Step 4: Normalize avg_x and avg_y
    scaler = MinMaxScaler()
    m[['avg_x', 'avg_y']] = scaler.fit_transform(m[['avg_x', 'avg_y']])

    # Update the dictionary
    df_dict[i] = m

In [119]:
#dictionary for the graphs
match_graph_dict = {}
#iterate through the dfs
for match in df_dict.values():

    match_id = match['match_id'].iloc[0]
    max_minute = match['minute'].max()
    #iterate from minute 0 to 15 minutes before end of game (it will not include the last 5 minutes, only the targets), and iterate every 5 minutes
    for start_minute in range(0, 85, 5):
        if start_minute == 40:
            pass
        else:
            end_minute = start_minute + 4 
            if start_minute < 44:
                period = 1
            else:
                period = 2
            interval_df = match[(match['minute'] >= start_minute) & (match['minute'] <= end_minute) & (match['period'] == period)]
            graph = nx.DiGraph()
            edges = []

        #iterate through the passings of the match
        for node in pd.concat([interval_df['position'], interval_df['pass_recipient_position']], axis=0):
            if str(node) not in list(graph.nodes):
                pos_x =  match.loc[match['position'] == node,"avg_x"].iloc[0]
                pos_y =  match.loc[match['position'] == node,"avg_y"].iloc[0]
                graph.add_node(str(node),features = np.array([pos_x, pos_y]))

        for passing in interval_df[['position', 'pass_recipient_position']].itertuples():
            edge = (str(passing.position), str(passing.pass_recipient_position))
            edges.append(edge)

        #count the frequencies
        edges_counter = Counter(edges)

        edge_and_count = [(edge[0], edge[1], edges_counter[edge]) for edge in edges]

        #add weighted edges to graph
        if edge_and_count:
            graph.add_weighted_edges_from(edge_and_count)

        #add finished graph to dictionary. add 1 minute to the end of the time window to match the minute the prediction window starts (so the ids match)
        match_graph_dict[str(match_id)+str(end_minute+1)] = graph

In [120]:
targets = pd.read_csv("C:/Users/thors/Documents/GitHub/5-semester/momentum_data.csv")

In [121]:
ids = targets["id"].to_list()
momentums = targets["momentum"].to_list()

#join the momentum value on each graph, matching the "match_id + time_interval" value
for i in ids:
    for key, value in match_graph_dict.items():
        if key == str(i):
            index = ids.index(i)
            value.graph["momentum"] = momentums[index]

In [122]:
match_graph_dict['38953025']
for node, data in graph.nodes(data=True):
    print(f"Node: {node}, Features: {data.get('features')}")

Node: Right Center Back, Features: [0.40995268 0.83633132]
Node: Left Center Back, Features: [0.37577951 0.19785105]
Node: Center Back, Features: [0.3524219  0.56735406]
Node: Center Defensive Midfield, Features: [0.35270904 0.4579695 ]
Node: Right Wing Back, Features: [0.67930475 0.91179797]
Node: Right Center Forward, Features: [0.57427948 0.61344976]
Node: Goalkeeper, Features: [0.         0.43467819]
Node: Left Wing Back, Features: [0.56614998 0.        ]
Node: Left Center Midfield, Features: [0.52749336 0.41509188]
Node: Right Center Midfield, Features: [0.69539249 0.75980712]
Node: Left Center Forward, Features: [1. 1.]


In [123]:
graph.nodes

NodeView(('Right Center Back', 'Left Center Back', 'Center Back', 'Center Defensive Midfield', 'Right Wing Back', 'Right Center Forward', 'Goalkeeper', 'Left Wing Back', 'Left Center Midfield', 'Right Center Midfield', 'Left Center Forward'))

In [124]:
import pickle
with open("Momentum graphs Bayern only.pkl", "wb") as file:
    pickle.dump(match_graph_dict, file)